In [33]:
import requests
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump

In [34]:
DATA_FILE = 'crawler/coinmarketExample.csv'

data = pd.read_csv(DATA_FILE)
data.describe()

,date,open,high,low,close,volume,market cap
count,2188,2188,2188,2188,2188,2188,2188
unique,2188,2162,2157,2169,2158,1946,2187
top,"Nov 14, 2017",418.42,108.00,429.08,104.00,0,"1,229,098,150"
freq,1,3,3,3,4,243,2


In [35]:
print(data['open'])

0       5,571.51
1       5,399.37
2       5,312.49
3       5,335.88
4       5,304.16
          ...   
2183      116.38
2184      139.00
2185      144.00
2186      134.44
2187      135.30
Name: open, Length: 2188, dtype: object


## Tentar substituir todos os valores de market cap por 2 SE o valor na tabela for maior que 2500 e 1 caso contrario

In [36]:
for idx, class in enumerate(data['market cap']):
    if (("2" in class)):
        data['market cap'][idx] = 0
    else:
        data['market cap'][idx] = 1

SyntaxError: invalid syntax (<ipython-input-36-a27fcf96727a>, line 1)

## Passar tudo pra inteiro depois

In [37]:
if (not data['open'].dtype == np.dtype(int)):
    data['open'] = data['open'].str.extract('(\d+)').astype(int)
if (not data['high'].dtype == np.dtype(int)):
    data['high'] = data['high'].str.extract('(\d+)').astype(int)    
if (not data['low'].dtype == np.dtype(int)):
    data['low'] = data['low'].str.extract('(\d+)').astype(int)
if (not data['close'].dtype == np.dtype(int)):
    data['close'] = data['close'].str.extract('(\d+)').astype(int)
if (not data['volume'].dtype == np.dtype(int)):
    data['volume'] = data['volume'].str.extract('(\d+)').astype(int)
if (not data['market cap'].dtype == np.dtype(int)):
    data['market cap'] = data['market cap'].str.extract('(\d+)').astype(int)

In [38]:
data.describe()

,open,high,low,close,volume,market cap
count,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000
mean,270.539305,270.557130,268.684186,270.452468,55.186015,42.719378
std,269.336997,270.440956,266.272336,269.328312,130.142937,76.055426
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,6.000000,6.000000,6.000000,6.000000,4.000000,4.000000
50%,233.000000,235.000000,232.000000,233.000000,16.000000,9.000000
75%,452.250000,455.000000,446.250000,452.250000,50.000000,67.000000
max,998.000000,998.000000,996.000000,999.000000,994.000000,988.000000


## Comeca o treinamento

In [39]:
X = data[['open', 'high', 'low', 'close', 'volume']].values
Y = data['market cap'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

X sao as variaveis que vou usar pra chegar ate Y, no exemplo dele ele usa temp, wind e humidity pra chegar ate weater

In [40]:
knn_clf = KNeighborsClassifier(n_neighbors = 5)
score = cross_val_score(knn_clf, X, Y, cv = 8)

print("CV: {}".format(np.mean(score)))

knn_clf.fit(X_train, Y_train)
confusion = confusion_matrix(y_true = Y_test, y_pred = knn_clf.predict(X_test))
print('Confusion matrix:')
print(confusion)

CV: 0.5817625197187241
Confusion matrix:
[[30  0  0 ...  0  0  0]
 [ 0  6  2 ...  0  0  0]
 [ 0  0 44 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  1  0]]


Faz o dump do joblib depois que terminar de treinar tudo

In [41]:
dump(knn_clf, 'service/knn_clf.joblib')

['service/knn_clf.joblib']